# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Dec 15th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

In [2]:
# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd

# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd
/data/pt_02586/sub_01/preprocessing


: 1

In [3]:
BVEC=$SUBJECTDIR'preprocessing/bvecs.txt';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';

BVEC=$SUBJECTDIR'preprocessing/bvecs.txt';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';


: 1

### Creating Brain Mask

#### What are we correcting?
We want to separate out the brain from the skull, surrounding tissue and background. 
#### Why does this happen?
Many step perform better when we are only focusing on the brain rather than the full head and background. 
#### What if we ignore it?
Various distorsion correction step will underperform or fail. 
#### Why correct it now?
After denoising and degibbsing, we have sharper boundaries and we need to mask before moving on to distorsion correction. 
#### correction tools: dwiextract, mcflirt, N4BiasFieldCorrection, bet
In this step, we will extract the b0s images from the full data using Mrtrix's dwiextract (this could be done with any tool that can select specific volumes out of the full data).  
Then, we will use FSL's mcflirt to do some quick subject motion correction between the b0s (since they are typically aquired spread out in time).  
We will average together the motion corrected b0s for maximal SNR.  
Then, we will use ANTS' N4BiasFieldCorrection to artificially make the image more homogeonous in intensity, this will really help the brain mask tool since it work with image intensity gradients and threshold.  
Finally, we will use FSL's BET (brain extraction tool) to create a mask of the brain.
#### Quality control
This step is heavy on manual quality control.  
- After extracting the b0s, you should manually look through them to make sure there isnt a broken one, if so discard it.  
- After motion correcting the b0s, you should make sure they all line up, otherwise the averaging step will fail.  
- We apply the N4 image homogenisation step multiple times, "as much as needed or until it doesnt change the image anymore", this need to be decided by QC.  
- Finally, the BET requires a threshold parameter (between 0 and 1) that need to be tune by hand, typically we compute multiple BET with different threshold and we find the right value by elimination; if the mask cuts part of the brain, make the threshold smaller, if the the mask still contain piece of skull and other, make the threshold bigger.

#### This threshold can be different for different subject.  

dwiextract can extract 3D volumes from diffusion data based on bvec/bval  
We only need the "-bzero" flag here to extract all b0s

In [ ]:
# We extract the b0 images from
# the denoised and degibbs data
# using dwiextract and the bval table
# dwiextract can do more but the -bzero flag is what we need here
dwiextract $SUBJECTDIR'preprocessing/dwi_denoise_degibbs.nii.gz' \
           $SUBJECTDIR'preprocessing/b0_denoise_degibbs.nii.gz' 
           -bzero \
           -fslgrad $BVEC $BVAL;

mcflirt corrects for motion by doing a simple linear registration.  
We use the default tool which is a 6-dof (degree of freedom) registration (can be modified with the '-dof' flag)  
We specify to which volume the others should be registered with the "-refvol" (reference volume) flag.  
Most of time you will not care and use the first volume, if many volume are a bit messy you might want to chose the cleanest one has the reference.

In [ ]:
# mcflirt stands for Motion Correction flirt
# flirt is the linear registration FSL tool
# (FMRIB's Linear Image Registration Tool)
fsl5.0-mcflirt -in $SUBJECTDIR'preprocessing/b0_denoise_degibbs.nii.gz' \
              -out $SUBJECTDIR'preprocessing/b0_denoise_degibbs_moco.nii.gz' \
              -stages 4 \
              -refvol 0;


In [ ]:
# We average all the motion corrected volume to get
# a single 3D image
mrmath $SUBJECTDIR'preprocessing/b0_denoise_degibbs_moco.nii.gz' \
       mean \
       $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco.nii.gz' \
       -axis 3;


N4BiasFieldCorrection (or simply N4 bias correction) is an improvement of N3 (nonparameteric nonuniform normalization) bias correction.  
It makes image more homogeneous in space.  
We apply it many times until it converges (i.e. stops changing the image when reaplied)  
We only want the create those artificially homogeonous images because it it easier to delimitate boundaries on them, we dont actually want to keep and use those images, only the mask they will produce.

In [ ]:
N4BiasFieldCorrection -d 3 \
     -i $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco.nii.gz' \
     -o $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x1.nii.gz';


In [ ]:
# 2 mores application in this case.
N4BiasFieldCorrection -d 3 -i $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x1.nii.gz' -o $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x2.nii.gz';
N4BiasFieldCorrection -d 3 -i $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x2.nii.gz' -o $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x3.nii.gz';

BET (brain extraction tool) is a tool for segmenting brain from background (technically we use bet2, but not one uses bet1 anymore and bet2 is just bet now).  
All the previous steps where simply used to provide a better input image to BET.  
The b0 is used because it has clear contrast, much high SNR than diffusion and it is in the same space has the diffusion (contrary to the T1 image for example).  
We motion corrected and averaged the b0s to get even more SNR.  
And we N4'd them many time to maximize uniformity.  

The '-m' flag generates the mask
The '-n' flag makes it NOT generate the image with the skull removed (because I want to use the mask on OTHER data, the one that wasen't N4'd and the non-b0s volume)  
The '-R' flag makes the bet "robust", it runs multiple time the estimation of the center of the brain for better accuracy.  
The '-f 0.1' flag is critical parameter between 0 and 1 that need tuning, smaller numbers give bigger mask.  
We will compute it with values 0.1, 0.3, 0.5, and 0.7 and select the best of them, i/e the one that doesn't include random pieces of skull and doesn't cut part of the brain.

In [ ]:
bet $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x3.nii.gz' \
    $SUBJECTDIR'preprocessing/b0_avg_N4x3_bet_init_0p1.nii.gz' \
    -m \
    -n \
    -f 0.1 \
    -R;

In [ ]:
# 3 more times with different f threshold
bet $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x3.nii.gz' $SUBJECTDIR'preprocessing/b0_avg_N4x3_bet_init_0p3.nii.gz' -m -n -f 0.3 -R;
bet $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x3.nii.gz' $SUBJECTDIR'preprocessing/b0_avg_N4x3_bet_init_0p5.nii.gz' -m -n -f 0.5 -R;
bet $SUBJECTDIR'preprocessing/b0_avg_denoise_degibbs_moco_N4x3.nii.gz' $SUBJECTDIR'preprocessing/b0_avg_N4x3_bet_init_0p7.nii.gz' -m -n -f 0.7 -R;

In [ ]:
# you can write to a textfile that best choice to refer to it later
read -p 'th: ' BETFOREDDY;
touch betthforeddy.txt;
echo $BETFOREDDY > betthforeddy.txt;
# or you can copy/rename the best mask to some uniform name

#### All this work was to create a good mask for the next distorsion correction step, we will want to recompute a new mask after the distorsion correction, i.e. a better non distorted mask

#### Also, since the data currently HAS distorsion, the mask CANNOT be too tight and perfect of it will cut out part of some volume, depending how distortedthere are and in which "direction" that distortion is, so in practice, the ACTUAL mask that will be used for the distorsion correction will be dilated version of the computed mask, i.e. we will add 1-2 voxels all around it to account for potential distortions

- check bvec flip  
- check weirdness in data

In [ ]:
# DTI/FA on non-eddy corrected data
dwi2tensor $SUBJECTDIR'preprocessing/dwi_denoise_degibbs.nii.gz' $SUBJECTDIR'preprocessing/dti_quick.nii.gz' -fslgrad $BVEC $BVAL;
tensor2metric $SUBJECTDIR'preprocessing/dti_quick.nii.gz' -fa $SUBJECTDIR'preprocessing/fa_quick.nii.gz';